In [2]:
import geopandas as gpd
import pandas as pd 
from shapely.geometry import Point
#import pysal as ps
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
bike = pd.read_csv('cartodb-query.csv')
bike.head()

,the_geom,end_stat_1,gender,bikeid,end_statio,start_st_1,tripdurati,field_1,usertype,cartodb_id,start_st_2,stoptime,birth_year,end_stat_3,end_stat_2,start_st_3,start_stat,starttime
0,NaN,DeKalb Ave & S Portland Ave,1,19838,241,324,169,2065,Subscriber,2066,DeKalb Ave & Hudson Ave,1/20/2016 23:34:37,1942.0,-73.974931,40.689810,-73.981013,40.689888,2016-01-20
1,NaN,Emerson Pl & Myrtle Ave,2,15454,354,243,647,3038,Subscriber,3039,Fulton St & Rockwell Pl,2/4/2016 19:26:10,1995.0,-73.962236,40.693631,-73.979382,40.688226,2016-02-04
2,NaN,E 10 St & Avenue A,1,22147,445,467,1918,4527,Subscriber,4528,Dean St & 4 Ave,2/20/2016 20:44:27,1984.0,-73.981420,40.727408,-73.978951,40.683125,2016-02-20
3,NaN,Stanton St & Chrystie St,1,22487,265,467,1939,5184,Subscriber,5185,Dean St & 4 Ave,2/28/2016 17:03:27,1983.0,-73.991475,40.722293,-73.978951,40.683125,2016-02-28
4,NaN,Kent Ave & S 11 St,1,21919,282,83,1081,5498,Subscriber,5499,Atlantic Ave & Fort Greene Pl,3/10/2016 16:51:55,1983.0,-73.968341,40.708273,-73.976323,40.683826,2016-03-10


In [29]:
bikesub = bike[['cartodb_id','start_st_1','end_statio','end_stat_3','end_stat_2','start_st_3','start_stat']]
bikesub.columns = ['id','start_id','end_id','end_long','end_lat','start_long','start_lan']

In [30]:
bikesub.head()

,id,start_id,end_id,end_long,end_lat,start_long,start_lan
0,2066,324,241,-73.974931,40.689810,-73.981013,40.689888
1,3039,243,354,-73.962236,40.693631,-73.979382,40.688226
2,4528,467,445,-73.981420,40.727408,-73.978951,40.683125
3,5185,467,265,-73.991475,40.722293,-73.978951,40.683125
4,5499,83,282,-73.968341,40.708273,-73.976323,40.683826


In [37]:
st_end = pd.DataFrame(bikesub.groupby(['start_id','end_id'],as_index=False).size())
st_end.reset_index(inplace=True)
st_end.columns = ['start_id','end_id','Count']
st_end.sort_values('Count',ascending=False).head(10)

,start_id,end_id,Count
2468,324,366,1072
2412,324,270,1057
2398,324,241,904
1028,258,324,684
2405,324,258,532
2408,324,262,527
713,243,270,516
2401,324,244,513
3008,384,244,493
2500,324,419,488


In [21]:
start = pd.DataFrame(bikesub.groupby(['start_id','start_long','start_lan'],as_index=False).size())
start.reset_index(inplace=True)
start.columns = ['start_id','start_long','start_lan','Count']
start.head()

,start_id,start_long,start_lan,Count
0,83,-73.976323,40.683826,7273
1,241,-73.974931,40.689810,3074
2,243,-73.979382,40.688226,5033
3,258,-73.968855,40.689407,4228
4,262,-73.973730,40.691782,2237


In [28]:
start.sort_values('Count',ascending=False)

,start_id,start_long,start_lan,Count
8,324,-73.981013,40.689888,9990
15,467,-73.978951,40.683125,7276
0,83,-73.976323,40.683826,7273
10,384,-73.964915,40.683048,6096
31,3429,-73.977908,40.685068,5551
2,243,-73.979382,40.688226,5033
5,274,-73.976682,40.686919,4954
11,395,-73.984106,40.688070,4867
3,258,-73.968855,40.689407,4228
13,416,-73.972652,40.687534,3894


In [8]:
#bikesub.dropna(axis=0, subset=['latitude','longitude'], inplace=True)
bikesub['start_lonlat'] = list(zip(bikesub.start_long, bikesub.start_lan))
#bikesub['start_lonlat'] = list(zip(bikesub.start_long, linkNYC.start_lan))
bikesub['geometry'] = bikesub[['start_lonlat']].applymap(lambda x:Point(x))
bikesub.head()

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,end_long,end_lat,start_long,start_lan,start_lonlat,geometry
0,2066,-73.974931,40.689810,-73.981013,40.689888,"(-73.981013, 40.689888)",POINT (-73.981013 40.689888)
1,3039,-73.962236,40.693631,-73.979382,40.688226,"(-73.979382, 40.688226)",POINT (-73.979382 40.688226)
2,4528,-73.981420,40.727408,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
3,5185,-73.991475,40.722293,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
4,5499,-73.968341,40.708273,-73.976323,40.683826,"(-73.97632328, 40.68382604)",POINT (-73.97632328 40.68382604)


In [9]:
bikesub_start= gpd.GeoDataFrame(bikesub)
bikesub_start.crs = {'init': 'epsg:4326'} 
bikesub_start.head()

,id,end_long,end_lat,start_long,start_lan,start_lonlat,geometry
0,2066,-73.974931,40.689810,-73.981013,40.689888,"(-73.981013, 40.689888)",POINT (-73.981013 40.689888)
1,3039,-73.962236,40.693631,-73.979382,40.688226,"(-73.979382, 40.688226)",POINT (-73.979382 40.688226)
2,4528,-73.981420,40.727408,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
3,5185,-73.991475,40.722293,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
4,5499,-73.968341,40.708273,-73.976323,40.683826,"(-73.97632328, 40.68382604)",POINT (-73.97632328 40.68382604)


In [12]:
bikesub_start

,id,end_long,end_lat,start_long,start_lan,start_lonlat,geometry
0,2066,-73.974931,40.689810,-73.981013,40.689888,"(-73.981013, 40.689888)",POINT (-73.981013 40.689888)
1,3039,-73.962236,40.693631,-73.979382,40.688226,"(-73.979382, 40.688226)",POINT (-73.979382 40.688226)
2,4528,-73.981420,40.727408,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
3,5185,-73.991475,40.722293,-73.978951,40.683125,"(-73.97895137, 40.68312489)",POINT (-73.97895137 40.68312489)
4,5499,-73.968341,40.708273,-73.976323,40.683826,"(-73.97632328, 40.68382604)",POINT (-73.97632328 40.68382604)
5,5536,-73.999733,40.719105,-73.976323,40.683826,"(-73.97632328, 40.68382604)",POINT (-73.97632328 40.68382604)
6,6036,-74.004607,40.712733,-73.979382,40.688226,"(-73.979382, 40.688226)",POINT (-73.979382 40.688226)
7,6070,-73.973730,40.691782,-73.979382,40.688226,"(-73.979382, 40.688226)",POINT (-73.979382 40.688226)
8,6812,-73.973730,40.691782,-73.973730,40.691782,"(-73.9737299, 40.6917823)",POINT (-73.97372990000001 40.6917823)
9,7297,-73.983043,40.689622,-73.979677,40.686832,"(-73.9796772, 40.68683208)",POINT (-73.9796772 40.68683208)
